In [2]:
# Instalação das dependências
!pip install 'crewai[tools]'==0.105.0

  Using cached crewai-0.105.0-py3-none-any.whl.metadata (28 kB)
  Using cached auth0_python-4.8.1-py3-none-any.whl.metadata (9.0 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached instructor-1.7.4-py3-none-any.whl.metadata (19 kB)
  Using cached json_repair-0.39.1-py3-none-any.whl.metadata (11 kB)
  Using cached json5-0.10.0-py3-none-any.whl.metadata (34 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached litellm-1.60.2-py3-none-any.whl.metadata (36 kB)
  Using cached openai-1.66.2-py3-none-any.whl.metadata (25 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.30.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.30.0-py3-none-

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Adicionando a API da OpenAI às variáveis de ambiente
import os
# from google.colab import userdata

openai_api_key = os.getenv("OPENAI_API_KEY")
# os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [29]:
# Importando classes necessárias para o kick-off
from crewai import Agent, Task, Crew, Process

## Equipe proposta para geração de testes automatizados
#### Agentes:
- QA: Este agente irá analizar o código e definir os requisitos para o teste automatizado.
- Developer: Este agente irá desenvolver o código dos testes.
- Tester: Este agente irá testar e corrigir o código dos testes.

In [31]:
qa = Agent(
    role="Quality Analyst", # Definição do papel
    goal="Plan engaging and factually accurate content on {code}", # Objetivo do agente
    backstory="You're working on an important software company " # "História prévia" do agente, basicamente o seu prompt do sistema principal
              "and your role is to generate quality requirements "
              "to guide the developers through the automated tests "
              "development process. You must analyze the code thoroughly "
              "and map all the possibilities and requirements for the tests. "
              "You must also provide a desired coverage for the tests."
              "Your work is the basis for "
              "the Developer to create the code for the automated tests.",
    allow_delegation=False,
	  verbose=True
)

In [32]:
developer = Agent(
    role="Developer",
    goal="Write fail proof tests for the provided code: {code}",
    backstory="You're working on an important software company "
              "and your role is creating tests based on given requirements. "
              "Your work is critic for the company success, "
              "so you must work as hard as possible to achieve the desired coverage."
              "You will write the code that will be passed to the Tester. ",
    allow_delegation=False,
	  verbose=True
)

In [78]:
from typing import Type

from crewai.tools.tool_usage import BaseTool
from crewai_tools import CodeInterpreterTool

from pydantic import BaseModel, Field

class Code(BaseModel):
    code: str = Field(..., description="The code to be saved.")

class SaveCodeTool(BaseTool):
    name: str = "save_code"
    description: str = "Save the provided code."
    args_schema: Type[BaseModel] = Code

    def _run(self, code: Code):
      with open("tests.py", "w") as f:
        f.write(code)

In [79]:
tester = Agent(
    role="Tester",
    goal="Execute and correct the tests for the provided code: {code}",
    backstory="You're working on an important software company "
              "and your role is executing and correcting tests. "
              "Your work is critic for the company success, "
              "so you must work as hard as possible to pass the tests.",
    allow_delegation=False,
    allow_code_execution=True,
    allow_self_execution=True,
	  verbose=True,
    tools=[SaveCodeTool(), CodeInterpreterTool()]
  )

In [66]:
# Definição da task para o agente QA
analyze = Task(
    description=(
        "1. Thoroughly analyze the provided codebase to identify key functionalities, edge cases, and conditions that need testing.\n"
        "2. Map out possible test scenarios and edge cases, ensuring complete test coverage.\n"
        "3. Create detailed test requirements, specifying input, expected output, and execution conditions for each test.\n"
        "4. Estimate the desired test coverage for each functionality and module of the code.\n"
        "5. Document any special conditions, configurations, or dependencies required for test execution.\n"
        "6. Provide a final summary of test requirements, including goals for coverage and specific testing strategies.\n"
        "7. Ensure that the test plan is clear, feasible, and aligned with the development process."
    ),
    expected_output="A detailed test requirements document, including test scenarios, coverage goals, and special conditions for test execution.",
    agent=qa,
)


In [67]:
# Definição da task para o agente Developer
develop = Task(
    description=(
        "1. Review the provided test requirements document, understanding the test scenarios and coverage goals.\n"
        "2. Analyze the given code to identify potential weak points and areas that require rigorous testing.\n"
        "3. Write automated test cases to cover all identified scenarios, ensuring the tests are robust and fail-proof.\n"
        "4. Ensure tests include both positive and negative cases, boundary conditions, and edge cases.\n"
        "5. Write clear, concise, and maintainable test code with proper assertions and logging.\n"
        "6. Ensure that tests are easy to execute and can be integrated into the CI/CD pipeline.\n"
        "7. Review the coverage to ensure it aligns with the desired goals specified in the test requirements document.\n"
        "8. Provide detailed documentation for the tests, including execution steps and expected results."
    ),
    expected_output="A set of automated, fail-proof tests that cover all specified scenarios, with clear documentation and integration steps.",
    agent=developer,
)


In [81]:
test = Task(
    description=(
        "1. Receive the test cases from the Developer.\n"
        "2. Execute the provided test cases against the codebase.\n"
        "3. Capture the output of the test execution.\n"
        "4. Document any errors, failures, or inconsistencies found during the execution.\n"
        "5. Ensure that all provided test cases are executed and tested thoroughly.\n"
        "6. Save the test execution results, including logs, output, and any failures.\n"
        "7. Don't forget to save the test code.\n"
        "8. Return the execution results along with a detailed report of any issues or bugs found during testing.\n"
        "9. Provide recommendations for fixes or improvements based on the test results."
    ),
    expected_output=(
        "A detailed report including the output from the executed test cases, logs, any errors found, and "
        "recommendations for fixes or improvements based on the execution results."
    ),
    agent=tester,
)


In [82]:
crew = Crew(
    agents=(qa, developer, tester),
    tasks=(analyze, develop, test),
    process=Process.sequential,
    verbose=True
)

In [83]:
code = "def divide(a, b):"
"   if b == 0:"
"       raise ValueError('Division by zero is not allowed.')"
"   return a / b"
result = crew.kickoff(inputs={"code": code})

 
[2025-03-12 14:44:05][🚀 CREW 'CREW' STARTED, 38711EED-502E-4155-9E62-5DF032CC2694]: 2025-03-12 14:44:05.175602
 
[2025-03-12 14:44:05][📋 TASK STARTED: 1. THOROUGHLY ANALYZE THE PROVIDED CODEBASE TO IDENTIFY KEY FUNCTIONALITIES, EDGE CASES, AND CONDITIONS THAT NEED TESTING.
2. MAP OUT POSSIBLE TEST SCENARIOS AND EDGE CASES, ENSURING COMPLETE TEST COVERAGE.
3. CREATE DETAILED TEST REQUIREMENTS, SPECIFYING INPUT, EXPECTED OUTPUT, AND EXECUTION CONDITIONS FOR EACH TEST.
4. ESTIMATE THE DESIRED TEST COVERAGE FOR EACH FUNCTIONALITY AND MODULE OF THE CODE.
5. DOCUMENT ANY SPECIAL CONDITIONS, CONFIGURATIONS, OR DEPENDENCIES REQUIRED FOR TEST EXECUTION.
6. PROVIDE A FINAL SUMMARY OF TEST REQUIREMENTS, INCLUDING GOALS FOR COVERAGE AND SPECIFIC TESTING STRATEGIES.
7. ENSURE THAT THE TEST PLAN IS CLEAR, FEASIBLE, AND ALIGNED WITH THE DEVELOPMENT PROCESS.]: 2025-03-12 14:44:05.185058
 
[2025-03-12 14:44:05][🤖 AGENT 'QUALITY ANALYST' STARTED TASK]: 2025-03-12 14:44:05.185970
# Agent: Quality Analy

In [84]:
# Resultado final do artigo
from IPython.display import Markdown
Markdown(result.raw)

Execution Results:
- All tests passed successfully, as indicated by the output:
```
----------------------------------------------------------------------
Ran 10 tests in 0.000s

OK
```

- The tests validated various division scenarios including:
  - Basic division of positive integers.
  - Division involving negative integers.
  - Division by zero, which correctly raised a `ZeroDivisionError`.
  - Division resulting in both integer and float outputs.
  - Precision checks for float division and recurring decimals.

Recommendations for fixes or improvements:
1. **Documentation**: Ensure that the documentation clearly specifies input requirements, especially for edge cases.
2. **Input Validation**: Currently, the function assumes valid numeric inputs. Adding explicit input validation to check types could prevent potential runtime errors.
3. **Exception Logging**: Instead of only raising exceptions, consider logging errors for monitoring and debugging purposes in a larger application context.
4. **Precision Handling**: Float comparisons can be tricky; ensure that there's an understanding of the precision requirements for any specific use case.

Overall, the `divide` function demonstrates robustness against varied test cases and appears to be well-implemented according to the defined test requirements.